# Demo for SPEL using LakeTemperature


In [1]:
%tb
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Built-in modules
import re        # Regular expressions
import importlib # MUST BE USED TO RELOAD MODULES
import sys 

# Importing modules for SPEL functions 
import utilityFunctions as uf
import edit_files as ef 
import fortran_modules as fm 
from analyze_subroutines import Subroutine

# mod_config : system configuration and static variables. 
# Where E3SM is stored, where unit tests are stored, etc.
from mod_config import default_mods, unittests_dir, scripts_dir, spel_mods_dir
from mod_config import ELM_SRC, spel_output_dir, _bc

No traceback available to show.


SPEL needs a "casename" (what your unit-test is called) and a list of subroutines to create a unit-test for "sub_name_list" (examples are "LakeTemperature", "SoilTemperature", etc...)


In [2]:
# Define Unit Test parameters 
import os 
casename = "SoilTemp"  # Name of the test case
case_dir = unittests_dir + casename # Directory to store the test case

# create directory for unit-test if it does not exist
if(not os.path.isdir(f"{case_dir}") ):
    print(f"Making case directory {case_dir}")
    if(not os.path.isdir(f"{unittests_dir}")):
        os.system(f"mkdir {unittests_dir}")
    os.system(f"mkdir {case_dir}")
    preprocess = True

# List of subroutines to be analyzed
sub_name_list = ["SoilTemperature"]
sub_name_list = [sub.lower() for sub in sub_name_list]

# variables needed for Unit Test
main_sub_dict = {}  # Dictionary to store all Subroutines in files needed for Unit Test
subroutines = {k.lower():[] for k in sub_name_list} # Dictionary for User Specified Subroutines

1st step for SPEL is determine which modules are needed for LakeTemperature and edit out the I/O, MPI, and other unneccessary modules. This must be somewhat tailored to ELM.

Currently if I module is not present in the "components/elm/src/" (ELM_SRC) nor in the "share/utils" directories (SHR_SRC), then the module and any dependency on it is automatically removed.

In [3]:
 # List to hold all the modules needed for the unit test
needed_mods = [] 
for s in sub_name_list:
    # Get general info of the subroutine
    subroutines[s] = Subroutine(s,calltree=['elm_drv'])

    # Process by removing certain modules and syntax
    # so that a standalone unit test can be compiled.
    fn = subroutines[s].filepath
    mod_dict, file_list = ef.process_for_unit_test(fname=fn,case_dir=case_dir,
                            mods=needed_mods,required_mods=default_mods, 
                            main_sub_dict=main_sub_dict,
                            overwrite=False,verbose=False)
    subroutines[s] = main_sub_dict[s]

Adding clumps to clump_type instances
Couldn't find shr_megan_mod in ELM or shared source -- adding to removal list
Couldn't find petscsys in ELM or shared source -- adding to removal list
Couldn't find petscvec in ELM or shared source -- adding to removal list
Couldn't find petscmat in ELM or shared source -- adding to removal list
Couldn't find petscts in ELM or shared source -- adding to removal list
Couldn't find petscdm in ELM or shared source -- adding to removal list
Couldn't find petscdmda in ELM or shared source -- adding to removal list
Couldn't find unstructuredgridtype in ELM or shared source -- adding to removal list
Couldn't find mathfuncmod in ELM or shared source -- adding to removal list
Couldn't find tracer_varcon in ELM or shared source -- adding to removal list
Couldn't find fatesinterfacetypesmod in ELM or shared source -- adding to removal list
Adding filter to clumpfilter instances
Adding filter_inactive_and_active to clumpfilter instances
check_cpp_line::Skippin

In [4]:
# Create dictionary containing all identified user defined types
# 'type-name' : 'DerivedType Object'
type_dict = {}
for modname, mod in mod_dict.items():
    for utype, dtype in mod.defined_types.items():
        type_dict[utype] = dtype

SPEL can print out a module tree showing how the modules are linked for the unit-test subroutines
- The full tree can be pretty difficult to parse, having a cutoff depth is recommended.
- Every module uses 'shr_kind_mod' so that could be suppressed as well.

In [5]:
modtree = fm.print_spel_module_dependencies(mod_dict=mod_dict,subs=subroutines)

# Print the module tree
arrow = "-->"
cutoff_depth = 10 # Only print modules up to this depth
suppress_mod_list = ['shr_kind_mod']
for m in modtree:
    depth = m['depth']
    modname = m['module']
    if(modname in suppress_mod_list):
        continue
    if(depth == 1):
        print(_bc.HEADER + arrow*depth + modname + _bc.ENDC)
    elif(depth <= cutoff_depth):
        print( arrow*depth + modname)

soiltemperaturemod
-->decompmod
-->-->elm_varctl
-->-->elm_varcon
-->-->-->shr_const_mod
-->-->-->elm_varpar
-->-->-->-->elm_varctl
-->-->-->elm_varctl
-->-->domainmod
-->-->-->elm_varctl
-->elm_varctl
-->elm_varcon
-->-->shr_const_mod
-->-->elm_varpar
-->-->-->elm_varctl
-->-->elm_varctl
-->urbanparamstype
-->-->decompmod
-->-->-->elm_varctl
-->-->-->elm_varcon
-->-->-->-->shr_const_mod
-->-->-->-->elm_varpar
-->-->-->-->-->elm_varctl
-->-->-->-->elm_varctl
-->-->-->domainmod
-->-->-->-->elm_varctl
-->-->elm_varctl
-->-->elm_varcon
-->-->-->shr_const_mod
-->-->-->elm_varpar
-->-->-->-->elm_varctl
-->-->-->elm_varctl
-->-->landunittype
-->-->-->elm_varcon
-->-->-->-->shr_const_mod
-->-->-->-->elm_varpar
-->-->-->-->-->elm_varctl
-->-->-->-->elm_varctl
-->-->gridcelltype
-->-->-->landunit_varcon
-->-->-->elm_varcon
-->-->-->-->shr_const_mod
-->-->-->-->elm_varpar
-->-->-->-->-->elm_varctl
-->-->-->-->elm_varctl
-->-->-->topounit_varcon
-->-->-->-->elm_varctl
-->-->-->-->elm_varcon
-->--

In [6]:
test_mod = mod_dict['columndatatype']
test_mod.display_info()

Module Name: columndatatype
Module Depedencies
used shr_kind_mod
used shr_const_mod
used elm_varpar
used elm_varcon
used elm_varctl
used ch4varcon
used pftvarcon
used soilorder_varcon
used landunit_varcon
used column_varcon
used decompmod
used cnstatetype
used cndecompcascadecontype
used columntype
used landunittype
used timeinfomod
used urbanparamstype
Variables:
real 0-D nfix_timeconst
column_energy_state 0-D col_es
column_water_state 0-D col_ws
column_carbon_state 0-D col_cs
column_carbon_state 0-D c13_col_cs
column_carbon_state 0-D c14_col_cs
column_nitrogen_state 0-D col_ns
column_phosphorus_state 0-D col_ps
column_energy_flux 0-D col_ef
column_water_flux 0-D col_wf
column_carbon_flux 0-D col_cf
column_carbon_flux 0-D c13_col_cf
column_carbon_flux 0-D c14_col_cf
column_nitrogen_flux 0-D col_nf
column_phosphorus_flux 0-D col_pf
User Types:
Derived Type: column_energy_state
from Mod: /home/mrgex/SPEL_Openacc/scripts/../../repo/E3SM/components/elm/src/data_types/ColumnDataType.F90
In

In [7]:
for f in file_list:
    base_fn = f.split('/')[-1]
    print(base_fn)

shr_kind_mod.F90
elm_varctl.F90
shr_const_mod.F90
elm_varpar.F90
elm_varcon.F90
domainMod.F90
decompMod.F90
LandunitType.F90
landunit_varcon.F90
topounit_varcon.F90
GridcellType.F90
column_varcon.F90
elm_varsur.F90
UrbanParamsType.F90
VegetationType.F90
atm2lndType.F90
ColumnType.F90
CanopyStateType.F90
SolarAbsorbedType.F90
CH4varcon.F90
pftvarcon.F90
SharedParamsMod.F90
FuncPedotransferMod.F90
RootBiophysMod.F90
domainLateralMod.F90
SoilStateType.F90
AnnualFluxDribbler.F90
EnergyFluxType.F90
TopounitType.F90
TopounitDataType.F90
LandunitDataType.F90
soilorder_varcon.F90
CNStateType.F90
CNDecompCascadeConType.F90
timeinfoMod.F90
ColumnDataType.F90
subgridAveMod.F90
SpeciesMod.F90
VegetationPropertiesType.F90
dynSubgridControlMod.F90
VegetationDataType.F90
perfMod_GPU.F90
BandDiagonalMod.F90
SoilTemperatureMod.F90
subgridMod.F90
filterMod.F90


In [8]:
for s in sub_name_list:
    # Parsing means getting info on the variables read and written
    # to by the subroutine and any of its callees
    subroutines[s].parse_subroutine(dtype_dict=type_dict,
                                    main_sub_dict=main_sub_dict,verbose=True)

_preprocess_file::Found pointer assignment at line 294
t_building              => lun_es%t_building                       , &
_preprocess_file::Found pointer assignment at line 295
xmf                     => col_ef%xmf                              , &
_preprocess_file::Found pointer assignment at line 296
xmf_h2osfc              => col_ef%xmf_h2osfc                       , &
_preprocess_file::Found pointer assignment at line 297
fact                    => col_es%fact                             , &
_preprocess_file::Found pointer assignment at line 298
c_h2osfc                => col_es%c_h2osfc                         , &
_preprocess_file::Found pointer assignment at line 300
begc                    =>    bounds%begc                          , &
_preprocess_file::Found pointer assignment at line 301
endc                    =>    bounds%endc                            &
_preprocess_file::Finished analyzing for soiltemperature


In [9]:
read_types  = [] 
write_types = []

for s in sub_name_list:
    subroutines[s].child_subroutines_analysis(dtype_dict=type_dict,
                                        main_sub_dict=main_sub_dict,verbose=True)
    print(_bc.OKGREEN+f"Derived Type Analysis for {subroutines[s].name}")
    print(f"Read-Only")
    for key in subroutines[s].elmtype_r.keys():
        print(key, subroutines[s].elmtype_r[key])
    print(f"Write-Only")
    for key in subroutines[s].elmtype_w.keys():
        print(key, subroutines[s].elmtype_w[key])
    print(f"Read-Write")
    for key in subroutines[s].elmtype_rw.keys():
        print(key, subroutines[s].elmtype_rw[key])
    print(_bc.ENDC)
    for key in subroutines[s].elmtype_r.keys():
        c13c14 = bool('c13' in key or 'c14' in key)
        if(c13c14):
            del subroutines[s].elmtype_r[key]
            continue
        if("_inst" in key):
           print(f"error: {key} has _inst")
           sys.exit(1)
        read_types.append(key)
        
    for key in subroutines[s].elmtype_w.keys():
        c13c14 = bool('c13' in key or 'c14' in key)
        if(c13c14):
            del subroutines[s].elmtype_w[key]
            continue
        if("_inst" in key):
            print(f"error: {key} has _inst")
            sys.exit(1)
        write_types.append(key)
        
    for key in subroutines[s].elmtype_rw.keys():
        c13c14 = bool('c13' in key or 'c14' in key)
        if(c13c14):
            del subroutines[s].elmtype_w[key]
            continue
        write_types.append(key)

Analyzing child subroutine: soilthermprop
_preprocess_file::Finished analyzing for soilthermprop
Analyzing child subroutine: computegroundheatfluxandderiv
_preprocess_file::Finished analyzing for computegroundheatfluxandderiv
Analyzing child subroutine: computeheatdifffluxandfactor
_preprocess_file::Finished analyzing for computeheatdifffluxandfactor
Analyzing child subroutine: solvetemperature
_preprocess_file::Finished analyzing for solvetemperature
Analyzing child subroutine: setmatrix
_preprocess_file::Finished analyzing for setmatrix
Analyzing child subroutine: setmatrix_soil
_preprocess_file::Finished analyzing for setmatrix_soil
Analyzing child subroutine: setmatrix_soilnonurban
_preprocess_file::Finished analyzing for setmatrix_soilnonurban
Analyzing child subroutine: setmatrix_soil_snow
_preprocess_file::Finished analyzing for setmatrix_soil_snow
Analyzing child subroutine: setmatrix_standingsurfacewater
_preprocess_file::Finished analyzing for setmatrix_standingsurfacewater
A

In [10]:
# Make sure physical properties types are read/written:
list_pp = ['veg_pp','lun_pp','col_pp','grc_pp','top_pp']

print("read_types:",read_types)
print("write_types:",write_types)

aggregated_elmtypes_list = []
for x in read_types:
    dtype_inst = x.split('%')[0]
    if(dtype_inst not in aggregated_elmtypes_list):
        aggregated_elmtypes_list.append(dtype_inst)    
for x in write_types:
    dtype_inst = x.split('%')[0]
    if(dtype_inst not in aggregated_elmtypes_list):
        aggregated_elmtypes_list.append(dtype_inst)

# for l in list_pp:
#     aggregated_elmtypes_list.append(l)
print("list of global vars:",aggregated_elmtypes_list)

read_types: ['lun_pp%urbpoi', 'urbanparams_vars%t_building_max', 'urbanparams_vars%t_building_min', 'col_pp%snl', 'col_pp%itype', 'col_pp%landunit', 'bounds%endl', 'bounds%begl', 'bounds%begc', 'bounds%endc', 'col_ws%frac_h2osfc', 'col_pp%z', 'col_pp%zi', 'col_es%t_ssbef', 'col_ws%frac_sno_eff', 'lun_pp%itype', 'urbanparams_vars%tk_wall', 'urbanparams_vars%tk_roof', 'urbanparams_vars%nlev_improad', 'urbanparams_vars%tk_improad', 'col_pp%dz', 'soilstate_vars%watsat_col', 'soilstate_vars%tkmg_col', 'soilstate_vars%tkdry_col', 'col_pp%nlevbed', 'urbanparams_vars%cv_wall', 'urbanparams_vars%cv_roof', 'urbanparams_vars%cv_improad', 'soilstate_vars%csol_col', 'col_pp%npfts', 'col_pp%pfti', 'veg_pp%landunit', 'veg_pp%topounit', 'veg_pp%gridcell', 'veg_pp%active', 'solarabs_vars%sabg_patch', 'veg_ef%dlrad', 'canopystate_vars%frac_veg_nosno_patch', 'col_es%emg', 'top_af%lwrad', 'veg_ef%eflx_sh_grnd', 'veg_wf%qflx_evap_soi', 'col_ef%htvp', 'veg_ef%eflx_sh_snow', 'veg_wf%qflx_ev_snow', 'veg_ef%ef

In [11]:
from UnitTestforELM import set_active_variables
instance_to_user_type = {}
elm_inst_vars = {}
for type_name, dtype in type_dict.items():
    if('bounds' in type_name): 
        continue
    if(not dtype.instances):
        print(f"Warning: no instances found for {type_name}")
        cmd = f'grep -rin -E "^[[:space:]]*(type)[[:space:]]*\({type_name}" {ELM_SRC}/main/elm_instMod.F90'
        output = sp.getoutput(cmd)
        print(f"output: {output}")
        if(output):
            output = output.split('\n')
            if(len(output) > 1):
                print(f"Warning: multiple instances found for {type_name}")
                print(output)
                sys.exit(1)
            line = output[0]
            line = line.replace('::','')
            line = line.split(':')
            
            decl = line[1].strip()
            decl = decl.split()
            var = decl[1]
            new_inst = Variable(type_name,var,subgrid='?',ln=0,dim=0,declaration='elm_instMod')
            dtype.instances.append(new_inst)
            elm_inst_vars[var] = dtype
        else:
            print(f"Warning: no instances found for {type_name}")
    for instance in dtype.instances:
        instance_to_user_type[instance.name] = type_name

dtype_info_list = []
    
for s in sub_name_list:
    set_active_variables(type_dict,instance_to_user_type,
                            subroutines[s].elmtype_r,dtype_info_list)
    set_active_variables(type_dict,instance_to_user_type,
                            subroutines[s].elmtype_w,dtype_info_list)
    set_active_variables(type_dict,instance_to_user_type,
                            subroutines[s].elmtype_rw,dtype_info_list)
    
print(dtype_info_list)

lun_pp%urbpoi
urbanparams_vars%t_building_max
urbanparams_vars%t_building_min
col_pp%snl
col_pp%itype
col_pp%landunit
bounds%endl
bounds%begl
bounds%begc
bounds%endc
col_ws%frac_h2osfc
col_pp%z
col_pp%zi
col_es%t_ssbef
col_ws%frac_sno_eff
lun_pp%itype
urbanparams_vars%tk_wall
urbanparams_vars%tk_roof
urbanparams_vars%nlev_improad
urbanparams_vars%tk_improad
col_pp%dz
soilstate_vars%watsat_col
soilstate_vars%tkmg_col
soilstate_vars%tkdry_col
col_pp%nlevbed
urbanparams_vars%cv_wall
urbanparams_vars%cv_roof
urbanparams_vars%cv_improad
soilstate_vars%csol_col
col_pp%npfts
col_pp%pfti
veg_pp%landunit
veg_pp%topounit
veg_pp%gridcell
veg_pp%active
solarabs_vars%sabg_patch
veg_ef%dlrad
canopystate_vars%frac_veg_nosno_patch
col_es%emg
top_af%lwrad
veg_ef%eflx_sh_grnd
veg_wf%qflx_evap_soi
col_ef%htvp
veg_ef%eflx_sh_snow
veg_wf%qflx_ev_snow
veg_ef%eflx_sh_soil
veg_wf%qflx_ev_soil
veg_ef%eflx_sh_h2osfc
veg_wf%qflx_ev_h2osfc
lun_pp%wtlunit_roof
lun_ef%eflx_wasteheat
lun_ef%eflx_heat_from_ac
lun_ef%

In [12]:
for sub in subroutines.values():
    tree = sub.calltree[2:]
    sub.analyze_calltree(tree,case_dir)

soiltemperature
|---->soilthermprop
|---->computegroundheatfluxandderiv
|---->computeheatdifffluxandfactor
|---->solvetemperature
|---->|---->setmatrix
|---->|---->|---->setmatrix_soil
|---->|---->|---->|---->setmatrix_soilnonurban
|---->|---->|---->setmatrix_soil_snow
|---->|---->|---->setmatrix_standingsurfacewater
|---->|---->|---->setmatrix_standingsurfacewater_soil
|---->|---->|---->setmatrix_soil_standingsurfacewater
|---->|---->|---->assemblematrixfromsubmatrices
|---->|---->banddiagonal
|---->|---->|---->dgbsv
|---->phasechangeh2osfc
|---->phasechange_beta


In [13]:
main_sub_dict['soiltemperature'].printSubroutineInfo()

Subroutine soiltemperature in /home/mrgex/SPEL_Openacc/scripts/../../repo/E3SM/components/elm/src/biogeophys/SoilTemperatureMod.F90 L149-704
   Variable Declaration: L184-231
Has Arguments: 
  bounds_type bounds 0-D  184
  integer num_nolakec 0-D  185
  integer filter_nolakec 1-D ? 186
  integer num_urbanl 0-D  187
  integer filter_urbanl 1-D ? 188
  atm2lnd_type atm2lnd_vars 0-D  189
  urbanparams_type urbanparams_vars 0-D  190
  canopystate_type canopystate_vars 0-D  191
  soilstate_type soilstate_vars 0-D  192
  solarabs_type solarabs_vars 0-D  193
  energyflux_type energyflux_vars 0-D  194
Local Arrays:
  integer jtop 1-D c 202
  real cv 2-D c 203
  real tk 2-D c 204
  real fn 2-D c 205
  real fn1 2-D c 206
  real sabg_lyr_col 2-D c 209
  real hs_top 1-D c 211
  logical cool_on 1-D l 212
  logical heat_on 1-D l 213
  real fn_h2osfc 1-D c 214
  real dz_h2osfc 1-D c 215
  real tvector_nourbanc 2-D c 216
  real tvector_urbanc 2-D c 217
  real tk_h2osfc 1-D c 218
  real dhsdT 1-D c 219